In [5]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt

In [6]:
model = YOLO("C:/Users/bryan/OneDrive/Documentos/Tesis/fall_detection/runs/classification/best.pt")

In [19]:
res = model('dataset/Merged_Dataset_for_classification/test/0/CAUCAFall_ars100017_0.png')
print(res[0].probs.top1)
for re in  res:
    print("----------------------------------")
    print(re)


image 1/1 C:\Users\bryan\OneDrive\Documentos\Tesis\fall_detection\yolo\dataset\Merged_Dataset_for_classification\test\0\CAUCAFall_ars100017_0.png: 224x224 ADL 1.00, Fall 0.00, 25.4ms
Speed: 190.1ms preprocess, 25.4ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
0
----------------------------------
ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'ADL', 1: 'Fall'}
obb: None
orig_img: array([[[  5,  10,  23],
        [  4,   9,  22],
        [  2,   7,  20],
        ...,
        [ 34,  48,  75],
        [ 48,  61,  89],
        [ 64,  78, 107]],

       [[  8,  13,  26],
        [  5,  10,  23],
        [  2,   7,  20],
        ...,
        [ 35,  50,  77],
        [ 46,  60,  88],
        [ 68,  83, 114]],

       [[ 11,  15,  28],
        [  6,  11,  24],
        [  3,   7,  21],
        ...,
        [ 36,  52,  79],
        [ 43,  59,  87],
        [ 68,  84, 115]],

       ...,

       [[161, 165, 178]

In [24]:
cap = cv2.VideoCapture("C:/Users/bryan/OneDrive/Documentos/Tesis/GMASD/GMDCSA24-A-Dataset-for-Human-Fall-Detection-in-Videos/data/Subject 1/Fall/06.mp4")
video_fps = cap.get(cv2.CAP_PROP_FPS)
frame_rate = 6
frame_step = int(video_fps // frame_rate)
frame_idx = 0
model_yolo = YOLO("yolo11n.pt")
fall_counter = 0  # Contador de frames con caída
fall_threshold = 6  # Número de frames consecutivos necesarios

while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break

    if frame_idx % frame_step == 0:
        results = model_yolo(frame, classes=[0], conf=0.4, device="cpu", stream=True)
        fall_detected_in_frame = False  # Bandera para este frame

        for result in results:
            for box in result.boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                cropped = frame[y1:y2, x1:x2]

                if cropped.size == 0:
                    continue

                fall_res = model(cropped, classes=[1], conf=0.5, device="cpu")
                fall_detected = True if fall_res[0].probs.top1 == 1 else False

                if fall_detected:
                    fall_detected_in_frame = True

                label = "FALL" if fall_detected else "Normal"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255) if fall_detected else (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1), cv2.FONT_HERSHEY_SIMPLEX,
                            0.9, (0, 0, 255) if fall_detected else (255, 255, 255), 2)

        # Actualizar contador de caídas consecutivas
        if fall_detected_in_frame:
            fall_counter += 1
        else:
            fall_counter = 0

        if fall_counter >= fall_threshold:
            print("-------------------- CAIDA DETECTADA -------------------- ")
            fall_counter = 0  # Reiniciar contador después de detección

        frame_idx = 0
    frame_idx += 1

    cv2.imshow("YOLO Inference", frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

cap.release()
cv2.destroyAllWindows()


0: 384x640 1 person, 59.4ms

0: 224x224 ADL 0.99, Fall 0.01, 24.6ms
Speed: 5.4ms preprocess, 24.6ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)
Speed: 2.4ms preprocess, 59.4ms inference, 1.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 76.7ms

0: 224x224 ADL 1.00, Fall 0.00, 29.1ms
Speed: 8.7ms preprocess, 29.1ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Speed: 4.2ms preprocess, 76.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 63.4ms

0: 224x224 ADL 1.00, Fall 0.00, 26.7ms
Speed: 5.1ms preprocess, 26.7ms inference, 0.2ms postprocess per image at shape (1, 3, 224, 224)
Speed: 2.9ms preprocess, 63.4ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 person, 65.8ms

0: 224x224 ADL 0.93, Fall 0.07, 26.7ms
Speed: 5.1ms preprocess, 26.7ms inference, 0.1ms postprocess per image at shape (1, 3, 224, 224)
Speed: 3.8ms preprocess, 65.8ms inference, 1.

In [1]:
cap = cv2.VideoCapture("C:/Users/Invitado/Documents/fall_detection/data/GMDCSA24/Subject 4/Fall/08.mp4")
model_yolo = YOLO("yolo11n.pt")
while cap.isOpened():
    success, frame = cap.read()
    if not success:
        break
        # Run YOLO inference on the frame
    results = model_yolo(frame, classes=[0], conf = 0.5)
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
            cropped = frame[y1:y2, x1:x2]  # Recortar persona

            if cropped.size == 0:
                continue

            fall_res = model(cropped, classes=[1], conf = 0.5)
            fall_detected = any(r.boxes for r in fall_res)

            label = "FALL" if fall_detected else "Normal"

            # Dibujar la caja y el label
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255) if fall_detected else (0, 255, 0), 2)
            cv2.putText(frame, label, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX,
                        0.9, (0, 0, 255) if fall_detected else (255, 255, 255), 2)

    # Visualize the results on the frame
    frame = results[0].plot()
    # Display the annotated frame
    cv2.imshow("YOLO Inference", frame)
    # Break the loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
# Release the video capture object and close the display window
cap.release()
cv2.destroyAllWindows()

NameError: name 'cv2' is not defined

In [ ]:
frame = cv2.imread("C:/Users/Invitado/Documents/fall_detection/data/URFD/Fall/Fall01/Camera/fall-01-cam0-rgb-110.png")
results = model_yolo(frame, classes=[0])
for result in results:
    for box in result.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])  # Bounding box
        cropped = frame[y1:y2, x1:x2]  # Recortar persona

        if cropped.size == 0:
            continue

        fall_res = model(cropped, classes=[1])
        fall_detected = any(r.boxes for r in fall_res)

        label = "FALL" if fall_detected else "Normal"

        # Dibujar la caja y el label
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 255) if fall_detected else (0, 255, 0), 2)
        cv2.putText(frame, label, (x1, y1 - 40), cv2.FONT_HERSHEY_SIMPLEX,
                    0.9, (0, 0, 255) if fall_detected else (255, 255, 255), 2)

# Visualize the results on the frame
frame = results[0].plot()
# Display the annotated frame
plt.imshow(frame)
# Break the loop if 'q' is pressed